In [2]:
!python --version

Python 3.9.13


In [3]:
!pip3 install SpeechRecognition

!pip install pyaudio
!pip install pyttsx3

^C
^C


In [4]:
import pyaudio
import wave
import speech_recognition as sr
import pyttsx3 


In [5]:
# Initialize the recognizer
r = sr.Recognizer()
# Define a stream to record audio
p = pyaudio.PyAudio()

# Loop infinitely for user to speak

while(1):   
    try:
        with sr.Microphone(device_index=0) as source:
            # adjust threshold
            r.adjust_for_ambient_noise(source, duration=0.2)
             
            #listens for the user's input
            audio = r.listen(source)
             
            # Using google to recognize audio
            MyText = r.recognize_google(audio)
            MyText = MyText.lower()
 
            print("Did you say ",MyText)
            #SpeakText(MyText)
            
        
    except sr.RequestError as e:
        print("Could not request results; {0}".format(e))
         
    except sr.UnknownValueError:
        print("unknown error occurred")

result2:
[]
unknown error occurred


KeyboardInterrupt: 

In [6]:
from os import path

# Initialize the WAV recognizer
r = sr.Recognizer()
# Define a stream to record audio
p = pyaudio.PyAudio()


# obtain path to "test.wav" in the same folder as this script
AUDIO_FILE = path.join(path.dirname(path.realpath("__file__")), "test.wav")

try:
    with sr.AudioFile(AUDIO_FILE) as source:
        # adjust threshold
        r.adjust_for_ambient_noise(source, duration=0.2)
             
        #listens for the user's input
        audio = r.record(source)
             
        # Using google to recognize audio
        MyText = r.recognize_google(audio)
        MyText = MyText.lower()
 
        print("Did you say ",MyText)
        #SpeakText(MyText)
            
        
except sr.RequestError as e:
    print("Could not request results; {0}".format(e))
         
except sr.UnknownValueError:
    print("unknown error occurred")

result2:
{   'alternative': [   {   'confidence': 0.95835346,
                           'transcript': 'can you please import a style to '
                                         'protection area'},
                       {   'transcript': 'can you please import a style to '
                                         'protection earlier'},
                       {   'transcript': 'can you please import a style to '
                                         'protection are there'},
                       {   'transcript': 'can you please import a style to '
                                         'protection Alia'},
                       {   'transcript': 'can you please import a style to '
                                         'protection Aria'}],
    'final': True}
Did you say  can you please import a style to protection area


In [7]:
'''
# Function to convert text to speech
def SpeakText(text):
     
    # Initialize the engine
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()
'''     
     

'\n# Function to convert text to speech\ndef SpeakText(text):\n     \n    # Initialize the engine\n    engine = pyttsx3.init()\n    engine.say(text)\n    engine.runAndWait()\n'

In [8]:
!pip install pyspellchecker

In [9]:
import nltk
nltk.__version__

# stopwords include words that doesn't have emotion 
# so that we can drop this words while making sentiment analysis
from nltk.corpus import stopwords
#download stopwords module inside nltk data
nltk.download("stopwords")
nltk.download("wordnet")

import re 
import string
from spellchecker import SpellChecker

#import autocorrect
#from autocorrect import Speller


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\z004fuuz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\z004fuuz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
!pip install word2number

In [87]:
from collections import OrderedDict
from word2number import w2n


funcList = ["open", "create", "save", "delete", "show", "import","activate"]

def parser(transcript):
        
    indexDict = OrderedDict()
    funcEntityDict = OrderedDict()    
    wordList = transcript.split()
    
    #spelling correction 
    spell = SpellChecker()
    for i, word in enumerate(wordList):
        if word == "stl":
            wordList[i] = word
        else:
            wordList[i] = spell.correction(word)
    
    #corpus for deleting unnecessary words
    stop_words = nltk.corpus.stopwords.words("english")
    stop_words_list = ["please"]
    stop_words.extend(stop_words_list)
        
    asciiDict = {"left parenthesis" : "(",
                 "right parenthesis" : ")",
                 "comma" : ",",
                 "dash" : "-",
                 "semicolon" : ";",
                 "left square bracket" : "[",
                 "right square bracket" : "]",
                 "underscore" : "_"}

    for i, word in enumerate(wordList):
        if (word in funcList):
            indexDict[i] = word
            
    for i, key in enumerate(indexDict.keys()):
        if i == 0 and len(indexDict) == 1:
            funcEntityDict[indexDict.get(key)] = wordList[key+1:]
            break
        
        elif i == 0 and len(indexDict) != 1:
            lastKey = key
            pass
        
        elif (i == len(indexDict.keys())-1):
            funcEntityDict[wordList[lastKey]] = wordList[lastKey+1:key]
            funcEntityDict[wordList[key]] = wordList[key+1:]
            
        else:
            funcEntityDict[wordList[lastKey]] = wordList[lastKey+1:key]
            lastKey = key

    for i, (key, value) in enumerate(funcEntityDict.items()):

        if ((len(value) == 1) & (value[0] == "and")):
            
            funcEntityDict[key] = list(funcEntityDict.values())[i+1]
        
        value[:] = (item for item in value if item not in stop_words)
    

    for key, value in funcEntityDict.items():

        print(key, value)
        for i, word in enumerate(value):
            try:
                if (word in asciiDict):
                    value[i] = asciiDict.get(word)
                else:
                    value[i] = str(w2n.word_to_num(word))
            except ValueError as e:
                pass
    
        #whitespace on/off
        funcEntityDict[key] = " ".join(value)
    
    return funcEntityDict

def invoker(funcEntityDict):
    
    for key, value in funcEntityDict.items():

        if key == "open":
            openFunc(value)
        elif key == "create":
            createFunc(value)
        elif key == "save":
            saveFunc(value)
        elif key == "delete":
            deleteFunc(value)
        elif key == "show":
            showFunc(value)
        elif key == "import":
            importFunc(value)
        elif key == "activate":
            checkActivateSetup(key,value)
            importFunc(value)
        else:
            print("Function name error")

            
def checkActivateSetup(key,value):
    
    value_list = value.split(" ")
    setup_name = []
    mount_station_name = []
    index = 1
    
    #check if command type is given
    if value_list[0] == "setup":
        key_type = key+value_list[0] 
        #command is right, let's check if names exist
        if len(value_list) == 1:
            #no name is given so system should be checked and if one mount station and one setup is exists 
            #then they should be selected 
            setup_name_str = "0"
            mount_station_name_str = "0"

            print("no names given")
            
        else:
            for i,word in enumerate(value_list[1:]):
                print("first for", word, index)
                if word == "mount":
                    break
                else:
                    setup_name.append(word)
                    index = index + 1 
                    
            print("index",index)
            for i,word in enumerate(value_list[index:]):
                print("second for", word)
                mount_station_name.append(word) 
    else:
        print("Error: please provide an action name.")
        
    if len(setup_name) != 0:
        setup_name_str = "".join(setup_name)
        print(setup_name_str)
    else:
        setup_name_str = "0"
    if len(mount_station_name) != 0:
        mount_station_name_str = "".join(mount_station_name)
        mount_station_name_str = mount_station_name_str.replace("mountstation","MS")
        print(mount_station_name_str)
    else:
        mount_station_name_str = "0"

    return [key_type, setup_name_str, mount_station_name_str]
            
            
def openFunc(entity):
    print("Open: ", entity)
def createFunc(entity):
    print("Create: ", entity)
def saveFunc(entity):
    print("Save: ", entity)
def deleteFunc(entity):
    print("Delete: ", entity)
def showFunc(entity):
    print("Show: ", entity)
def importFunc(entity):
    print("Import: ", entity)



In [91]:
text = "can you please activate setup example underscore 01"

#delete all punctuations
#text_without_punctuation = "".join([char for char in text if char not in string.punctuation])

#stl correction
#print(re.findall("a style", text))
#clean_text = re.sub("a style", "stl", text)


#turns protection into protect, can be a problem
#ps = nltk.PorterStemmer()
#wn = nltk.WordNetLemmatizer()
#lemmatized_text = [ps.stem(word) for word in text_list]
#print(lemmatized_text)


invoker(parser(text))



activate ['setup', 'example', 'underscore', '01']
first for example 1
first for _ 2
first for 1 3
index 4
example_1
Import:  setup example _ 1
